This cell saves the annotated image with bounding boxes, labels, and confidence scores to the Noteable project. The path to the saved image is then displayed.

This cell uses the OpenCV library to draw rectangles around the detected objects in the image. Each rectangle is accompanied by a label indicating the type of object and its detection confidence score. The annotated image is then displayed.

This cell extracts the labels, pixel locations, and confidence scores of the detected objects and stores them in a pandas dataframe. The dataframe is then displayed, showing the label of each detected object, its bounding box coordinates, and the confidence score of the detection.

This cell installs the `opencv-python` package, which is a library for computer vision tasks. The code then imports necessary modules and reads an image from the specified path. The image's color channels are converted from BGR (Blue, Green, Red) to RGB (Red, Green, Blue) format. Finally, the image is displayed using the `matplotlib` library.

In [ ]:
!pip install -q opencv-python
import cv2
import matplotlib.pyplot as plt

# Read the image using OpenCV
image_path = 'fig/04Ge3YMUnWc.mp4-00049.jpg'
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image using matplotlib
plt.imshow(image_rgb)
plt.axis('off')
plt.show()

In [ ]:
!pip install -q transformers timm torch

This cell installs the `transformers`, `timm`, and `torch` packages. These libraries are essential for working with transformer models and deep learning in Python.

This cell imports the `DetrImageProcessor` and `DetrForObjectDetection` classes from the `transformers` library, as well as the `torch` library. These are used for object detection tasks using the DETR (DEtection TRansformer) model.

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch

In [ ]:
from PIL import Image
import requests

This cell imports the `Image` class from the `PIL` library and the `requests` module. The `Image` class is used for opening, manipulating, and saving different image file formats, while the `requests` module allows for making HTTP requests.

This cell fetches an image from a specified URL and opens it using the `Image` class from the `PIL` library. It then initializes the DETR image processor and model for object detection. The image is processed and passed to the model to detect objects. The detected objects, their locations (bounding boxes), and confidence scores are then printed.

In [ ]:



url = "http://images.cocodataset.org/val2017/000000039769.jpg"
url = "https://raw.githubusercontent.com/nils-holmberg/scom-dm/main/img/04Ge3YMUnWc.mp4-00049.jpg"
image = Image.open(requests.get(url, stream=True).raw)

processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

In [ ]:
import pandas as pd

# Extracting object detection labels and pixel locations
data = []
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    data.append([model.config.id2label[label.item()], box, round(score.item(), 3)])

# Creating a pandas dataframe
df = pd.DataFrame(data, columns=['Label', 'Pixel Location (x1, y1, x2, y2)', 'Confidence'])
df

In [ ]:
# Draw rectangles, labels, and confidence on the image using OpenCV
for _, row in df.iterrows():
    label, box, confidence = row['Label'], row['Pixel Location (x1, y1, x2, y2)'], row['Confidence']
    x1, y1, x2, y2 = box
    cv2.rectangle(image_rgb, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
    cv2.putText(image_rgb, f'{label}: {confidence:.2f}',
                (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Display the image with bounding boxes, labels, and confidence
plt.imshow(image_rgb)
plt.axis('off')
plt.show()

In [ ]:
# Save the annotated image to the Noteable project
output_image_path = 'fig/Annotated_Image.jpg'
cv2.imwrite(output_image_path, cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))
output_image_path